#Setup

In [49]:
!nvidia-smi

Wed Sep  7 05:57:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    32W / 250W |  13555MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [50]:
!pip install -q diffusers
!pip install -q transformers scipy ftfy
!pip install -q "ipywidgets>=7,<8"
!pip install -q gradio

In [52]:
import torch
!git clone https://github.com/sberbank-ai/Real-ESRGAN
!pip install -r Real-ESRGAN/requirements.txt
!wget https://huggingface.co/datasets/db88/Enhanced_ESRGAN/resolve/main/RealESRGAN_x4.pth -O /content/Real-ESRGAN/weights/RealESRGAN_x4.pth
%cd Real-ESRGAN
from realesrgan import RealESRGAN
device = torch.device('cuda')
realgan = RealESRGAN(device, scale = 4)
realgan.load_weights(f'/content/Real-ESRGAN/weights/RealESRGAN_x4.pth')
%cd /content/

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 46 (delta 13), reused 33 (delta 5), pack-reused 0
Unpacking objects: 100% (46/46), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-09-07 05:58:49--  https://huggingface.co/datasets/db88/Enhanced_ESRGAN/resolve/main/RealESRGAN_x4.pth
Resolving huggingface.co (huggingface.co)... 52.202.207.64, 52.6.16.131, 2600:1f18:147f:e850:db35:e0c7:187b:c770, ...
Connecting to huggingface.co (huggingface.co)|52.202.207.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/63/c5/63c5953a7844eb4bb3d4eb4d748940919ddc38416a10759d599d41773f03c627/aa00f09ad753d88576b21ed977e97d634976377031b178acc3b5b238df463400?response-content-disposition=attachment%3B%20filename%3D%22RealESRGAN_x4.pth%22 [following]
--2

In [53]:
from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [54]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=True).to('cuda')

#Make images

In [68]:
import numpy as np

def upscale(image):
    sr_image = realgan.predict(np.array(image))
    return sr_image

In [1]:
from torch import autocast

def make_img(prompt, width=512, height=512, num_imgs=1, cfg_scale=7.5, steps=50):
    imgs = []
    for _ in range(num_imgs):
        image = pipeline(prompt, width=width, height=height, scale=cfg_scale, steps=steps)["sample"][0]
        # image = upscale(image)
        imgs.append(image)
    return imgs

In [2]:
import gradio as gr

In [4]:
with gr.Blocks(title='Stable Diffusion Demo') as stable:
    with gr.Tabs() as tabs:
        with gr.TabItem('txt2img', id='txt2img_tab') as txt2img:
            with gr.Row():
                with gr.Column(scale=8):
                    prompt = gr.Textbox(label="Prompt") 
                with gr.Column(scale=1):
                    diffuse = gr.Button(value='Dream', label="Diffuse", elem_id='submit_btn')
            with gr.Row():
                height = gr.Slider(512, 1280, 512, step=64, label="Height")
                width = gr.Slider(minimum=512, maximum=1280, value=512, step=64, label="Width")
                num_imgs = gr.Slider(1, maximum=10, value=3, step=1, label="Number of Images")
                cfg_scale = gr.Slider(minimum=1, maximum=40, value=7.5, step=0.5, label="CFG_scale")
                steps = gr.Slider(minimum=1,maximum=150,value=50,step=1, label='Steps')
            gallery = gr.Gallery(label="Output", elem_id="txt2img_gallery_output").style(grid=3)
            diffuse.click(fn=make_img, inputs=[prompt, width, height, num_imgs, cfg_scale, steps], outputs=gallery)

stable.launch()

Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1624307adc0>, 'http://127.0.0.1:7861/', None)